In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
import os
from os.path import join, dirname
import pandas as pd

In [2]:
if os.name == 'nt': 
    os_name = 'nt' 
elif os.name == 'mac': 
    os_name = 'mac' 
else: 
    os_name = 'linux'
    
if os_name == 'nt': 
    driver_loc = "C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe" ##내 pc에 chroomdrirver 저장해놓은 경로 적어주기
else: 
    driver_loc = "C:/Users/HP/Downloads/chromedriver_win32/chromedriver.exe"

In [3]:
# 크롬 옵션 설정
option = webdriver.ChromeOptions()
option.add_argument('window-size=1920*1080')

#드라이버 등록
driver = webdriver.Chrome(driver_loc, options=option)

In [4]:
naver_login = 'https://nid.naver.com/nidlogin.login' 
cafe_url = 'https://cafe.naver.com/smartpolytech'

keyword_list = ['스프링', '웹']
result_dic = {}

for keyword in keyword_list :
    result_dic[keyword] = {
        'title' : [],
        'writer' : [],
        'href' : [],
        'date' : []
    }
    
exception_flag = 0 
exception_title_keyword_list = ['하둡','빅데이터'] 
exception_writer_keyword_list = []

In [5]:
#Login page 
driver.get(naver_login)

In [9]:
#webdriver로 열린 chrome창에 'id'라는 속성을 찾아서 'ID'라는  값을 입력해줌

login_id = 'j_i__'
login_pw =   'ggwldls2341!!'
# driver.find_element_by_name('id').send_keys(login_id)
# driver.find_element_by_name('pw').send_keys(login_pw)

In [7]:
#위에서 입력한 정보를 토대로 자동으로 Login 버튼을 클릭
#이 방법은 간단하지만 네이버 로그인 정책 바뀐 후로 자동입력 방지 문자를 입력해야한다는 번거로움 생김
# driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

In [10]:
#다른 login 방법(javascript 문법 이용)
driver.execute_script("document.getElementsByName('id')[0].value=\'"+login_id+"\'")
driver.execute_script("document.getElementsByName('pw')[0].value=\'"+login_pw+"\'")

In [11]:
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

In [12]:
#크롤링 할 url 접속
driver.get(cafe_url)
driver.implicitly_wait(3) #웹 지원 load를 위해 3초 기다려주도록

In [13]:
#크롤링 할 검색어 입력창 변수처리
search_input = driver.find_element_by_xpath('//*[@id="topLayerQueryInput"]')

In [14]:
#스크린샷
driver.get_screenshot_as_file('naver_main_headless.png')

True

In [15]:
#search_input 창에 위에서 적은 keyword 입력
search_input.send_keys(keyword)

In [16]:
#검색 버튼 클릭
search = driver.find_element_by_xpath('//*[@id="info-search"]/form/button')
search.click()
driver.implicitly_wait(3)

In [17]:
#iframe으로 이루어진 웹사이트의 frame을 cafe_main으로 맞춰주는 역할
iframe = driver.find_element_by_css_selector('iframe#cafe_main')

In [18]:
driver.switch_to.frame(iframe)

In [19]:
#게시글 보기 옵션 selectbox클릭
show_element = driver.find_element_by_xpath("""//*[@id="listSizeSelectDiv"]/a""")

In [20]:
show_element.click()

In [21]:
#50개씩 보기 클릭
show_50_element = driver.find_element_by_xpath("""//*[@id="listSizeSelectDiv"]/ul/li[7]/a""") 
show_50_element.click()

In [22]:
req = driver.page_source

In [23]:
html = BeautifulSoup(req,'html.parser')

In [24]:
find_div = html.find("div",{"class":"article-board result-board m-tcol-c"})

In [25]:
find_table = find_div.find("table").find("tbody")

In [27]:
find_tr = find_table.find_all("tr")

In [66]:
# find_href = find_tr.find("a",{"class":"article"}).attrs['href']

In [ ]:
find_title = find_tr.find("a",{"class":"article"}).text.strip().replace("[공지]\n                                \n                                \n\n                                \n                                \n                                    ","")

In [ ]:
find_title

In [65]:
# find_writer = find_tr.find("div",{"class":"pers_nick_area"})

In [64]:
# find_date = find_tr.find("td",{"class":"td_date"})

In [32]:
title_list = []
href_list = []
writer_list = []
date_list = []

for tr in find_tr :
    find_title = tr.find_all("a",{"class":"article"})
    find_writer = tr.find_all("div",{"class":"pers_nick_area"})
    find_date = tr.find_all("td",{"class":"td_date"})
    for title in find_title :
        title_list.append(title.text.strip().replace("[공지]\n                                \n                                \n\n                                \n                                \n                                    ",""))
        href_list.append(title.attrs['href'])
    for writer in find_writer :
        writer_list.append(writer.text.strip())
    for date in find_date :
        date_list.append(date.text)

result_dic['웹']['title'] = title_list
result_dic['웹']['href'] = href_list
result_dic['웹']['writer'] = writer_list
result_dic['웹']['date'] = date_list

result_dic

{'스프링': {'date': [], 'href': [], 'title': [], 'writer': []},
 '웹': {'date': ['2019.07.15.',
   '2019.07.10.',
   '2019.06.28.',
   '2019.04.16.',
   '2019.03.29.',
   '2019.01.01.',
   '2018.11.02.',
   '2018.09.29.',
   '2018.09.17.',
   '2018.07.16.',
   '2018.06.01.',
   '2018.05.22.',
   '2018.05.15.',
   '2018.05.09.',
   '2018.05.02.',
   '2018.04.18.',
   '2018.04.12.',
   '2018.04.10.',
   '2018.04.06.',
   '2018.03.31.'],
  'href': ['/ArticleRead.nhn?clubid=29316181&page=1&userDisplay=50&inCafeSearch=true&searchBy=0&query=%C0%A5&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=430&referrerAllArticles=true',
   '/ArticleRead.nhn?clubid=29316181&page=1&userDisplay=50&inCafeSearch=true&searchBy=0&query=%C0%A5&includeAll=&exclude=&include=&exact=&searchdate=all&media=0&sortBy=date&articleid=424&referrerAllArticles=true',
   '/ArticleRead.nhn?clubid=29316181&page=1&userDisplay=50&inCafeSearch=true&searchBy=0&query=%C0%A5&includeAll=&exclude=&include

In [43]:
import pandas as pd

In [55]:
result_data = pd.DataFrame(result_dic['웹'])

In [56]:
result_data = pd.DataFrame(result_data, columns=['title', 'writer','href','date'])

In [62]:
result_data.columns = ["title", "writer", "href", "date"]

In [63]:
result_data

,title,writer,href,date
0,[여름방학공지] 2019년도 2기생 여러분 여름방학 잘 보내시길 바랍니다.,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.07.15.
1,JAVA/ JSP/ 웹개발 / 소프트웨어개발자 신입,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.07.10.
2,[스프링부트] Session 처리,황원용교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.06.28.
3,자바 개발환경설정 (Java_home) 설정 및 시험문제 힌트,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.04.16.
4,[이러닝 공지] 한기대 교육컨텐츠 수강관련 안내,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.03.29.
5,[스마트금융과] 교육생 모집 웹페이지 접속자 분석,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.01.01.
6,[공지] 2019년도 스마트금융과 신입생 모집 (세부내용),김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.11.02.
7,데이터스트림즈 솔루션 매뉴얼 위치,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.09.29.
8,성한빈 면접후기 입니다.,2018성한빈,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.09.17.
9,여름방학 및 교수일정 공지,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.07.16.
